<a href="https://colab.research.google.com/github/thesamasaurus/kaldheim-ml-bot/blob/main/card_hash_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creating a Master Hash Table

### The output of this section allows us to take any url, onehot vector index, or card name, and change into another.  Master hash is a layered dictionary that in the following format:

master_hash["x_to_y"]["x"] 

### for example

master_hash["name_to_url"]["Doomskar Oracle"]

### this call will output the image url of Doomskar oracle.  You do not need to run this section for the bot to work, as the output is provided in the github.

In [ ]:
import os
import re
import pandas as pd
import copy
import numpy as np
import pickle
from google.colab import drive
import json

### Mount drive to access local files saved in google drive

In [ ]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### Load two API requests


In [ ]:
with open("/content/gdrive/MyDrive/Colab Notebooks/17_lands_scrapes/kaldheim/postman_saves/scryfall_collection_v2.postman_collection") as f:
  kald_data_1 = json.load(f)

In [ ]:
with open("/content/gdrive/MyDrive/Colab Notebooks/17_lands_scrapes/kaldheim/postman_saves/scryfall_collection_v2_p2.postman_collection") as f:
  kald_data_2 = json.load(f)

### Define functions to turn API requests into a hash table that can convert any datatype used in the project into any other

In [ ]:
def create_list_of_uris(scryfall_get):
  scryfall_get["data"][0]["image_uris"]["border_crop"]
  list_of_named_uris = []
  for i in range(0, len(scryfall_get["data"])):
    temp = []
    if ("image_uris" in scryfall_get["data"][i]):
      temp.append(scryfall_get["data"][i]["image_uris"]["border_crop"])
      temp.append(scryfall_get["data"][i]["name"])
      temp.append(scryfall_get["data"][i]["cmc"])
      temp.append(scryfall_get["data"][i]["color_identity"])
    else:
      temp.append(scryfall_get["data"][i]["card_faces"][0]["image_uris"]["border_crop"])
      temp.append(scryfall_get["data"][i]["name"])
      temp.append(scryfall_get["data"][i]["cmc"])
      temp.append(scryfall_get["data"][i]["color_identity"])
    list_of_named_uris.append(temp)
  return list_of_named_uris

### Taking output of scryfall api request, and creating master_hash

In [ ]:
list_of_named_uris_1 = create_list_of_uris(kald_data_1)
list_of_named_uris_2 = create_list_of_uris(kald_data_2)


In [ ]:
def add_list(master_list, named_uris):
  for i in named_uris:
    master_list.append(i)


In [ ]:
master_list = []
add_list(master_list, list_of_named_uris_1)
add_list(master_list, list_of_named_uris_2)

In [ ]:
def create_master_dict(url_name_array):
  master_dict = {}
  master_dict["url_to_name"] = {}
  master_dict["name_to_url"] = {}
  master_dict["name_to_cmc"] = {}
  master_dict["name_to_color"] = {}
  for i in range(len(url_name_array)):
    master_dict["url_to_name"][url_name_array[i][0]] = url_name_array[i][1]
  for i in range(len(url_name_array)):
    master_dict["name_to_url"][url_name_array[i][1]] = url_name_array[i][0]
  for j in range(len(url_name_array)):
    master_dict["name_to_cmc"][url_name_array[j][1]] = url_name_array[j][2]
    master_dict["name_to_color"][url_name_array[j][1]] = url_name_array[j][3]
  return master_dict

In [ ]:
master_hash = create_master_dict(master_list)

In [ ]:
for key in master_hash["name_to_url"]:
  if (isinstance(master_hash["name_to_url"][key], str)):
    master_hash["name_to_url"][key] = [master_hash["name_to_url"][key]]

### Appending Updated Postman Uris

#### Unfortunately, the scryfall API was not consistent, and images were being changed midway though the project.  For this reason, it became necessicary to add a second API request.

In [ ]:
with open("/content/gdrive/MyDrive/Colab Notebooks/17_lands_scrapes/kaldheim/postman_saves/response_3_1") as f:
  kald_data_3 = json.load(f)

In [ ]:
with open("/content/gdrive/MyDrive/Colab Notebooks/17_lands_scrapes/kaldheim/postman_saves/response_3_2") as g:
  kald_data_4 = json.load(g)

In [ ]:
list_of_named_uris_3 = create_list_of_uris(kald_data_3)
list_of_named_uris_4 = create_list_of_uris(kald_data_4)

In [ ]:
master_list_2 = []
add_list(master_list_2, list_of_named_uris_3)
add_list(master_list_2, list_of_named_uris_4)

In [ ]:
master_hash["name_to_url"]["Aegar, the Freezing Flame"]

['https://c1.scryfall.com/file/scryfall-cards/border_crop/front/6/8/68011f60-6202-48f4-8255-fb94764e2951.jpg?1610566556']

In [ ]:
for i in range(len(master_list_2)):
  if (master_hash["name_to_url"][master_list_2[i][1]][0] != master_list_2[i][0]):
    master_hash["name_to_url"][master_list_2[i][1]].append(master_list_2[i][0])

In [ ]:
master_hash["name_to_url"]["Aegar, the Freezing Flame"]

['https://c1.scryfall.com/file/scryfall-cards/border_crop/front/6/8/68011f60-6202-48f4-8255-fb94764e2951.jpg?1610566556',
 'https://c1.scryfall.com/file/scryfall-cards/border_crop/front/6/8/68011f60-6202-48f4-8255-fb94764e2951.jpg?1614989874']

### Another issue, was the some URLS being used in the draft were old and no longer shown by scryfall's API, so i needed to manually add them.

In [ ]:
master_hash["name_to_url"]["Brinebarrow Intruder"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/8/9/89960a74-9e17-4e30-874a-4286dd4c917d.jpg?1610998144", "https://c1.scryfall.com/file/scryfall-cards/border_crop/front/8/9/89960a74-9e17-4e30-874a-4286dd4c917d.jpg?1614476929"])
master_hash["name_to_url"]["Doomskar Oracle"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/6/d/6dae01c8-15bb-44ad-a2c6-9bcf7a9e8c17.jpg?1610586058", "https://c1.scryfall.com/file/scryfall-cards/border_crop/front/6/d/6dae01c8-15bb-44ad-a2c6-9bcf7a9e8c17.jpg?1614475433"])
master_hash["name_to_url"]["Ravenform"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/2/c/2c2cfcd7-b43a-4dad-a033-e661f55ceecd.jpg?1610586103", "https://c1.scryfall.com/file/scryfall-cards/border_crop/front/2/c/2c2cfcd7-b43a-4dad-a033-e661f55ceecd.jpg?1614477784"])
master_hash["name_to_url"]["Snow-Covered Swamp"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/6/a/6aa85af8-15f5-4620-8aea-0b45c28372ed.jpg?1610996926", "https://c1.scryfall.com/file/scryfall-cards/border_crop/front/9/1/9160baf7-5796-4815-8e9d-e804af70cb74.jpg?1610153618" ])
master_hash["name_to_url"]["Annul"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/4/b/4b1d4a59-11a0-4a55-8ac0-07377a9e6dc8.jpg?1610300632", "https://c1.scryfall.com/file/scryfall-cards/border_crop/front/4/b/4b1d4a59-11a0-4a55-8ac0-07377a9e6dc8.jpg?1614476636"])
master_hash["name_to_url"]["Master Skald"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/6/b/6b269e27-8e17-4bf2-b28a-a83ecc7d22c0.jpg?1611014236", "https://c1.scryfall.com/file/scryfall-cards/border_crop/front/6/b/6b269e27-8e17-4bf2-b28a-a83ecc7d22c0.jpg?1614475773"])
master_hash["name_to_url"]["Revitalize"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/3/a/3a9fb75e-c8e5-417b-83d4-5105af9c66c1.jpg?1611169934", "https://c1.scryfall.com/file/scryfall-cards/border_crop/front/3/a/3a9fb75e-c8e5-417b-83d4-5105af9c66c1.jpg?1614475898"])
master_hash["name_to_url"]["Mystic Reflection"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/f/7/f7b877e2-60eb-46cd-acd7-8555b9e7e993.jpg?1610656808",  'https://c1.scryfall.com/file/scryfall-cards/border_crop/front/f/7/f7b877e2-60eb-46cd-acd7-8555b9e7e993.jpg?1614477674'])
master_hash["name_to_url"]["Beskir Shieldmate"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/d/f/dfc1df84-9c47-444b-9d58-d9c7bed51c66.jpg?1610656522",  'https://c1.scryfall.com/file/scryfall-cards/border_crop/front/d/f/dfc1df84-9c47-444b-9d58-d9c7bed51c66.jpg?1614664042'])
master_hash["name_to_url"]["Littjara Kinseekers"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/c/1/c119f836-0707-49e2-b6d4-25f849d054a4.jpg?1610656790", "https://c1.scryfall.com/file/scryfall-cards/border_crop/front/c/1/c119f836-0707-49e2-b6d4-25f849d054a4.jpg?1614477561"])
master_hash["name_to_url"]["Starnheim Courser"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/5/c/5c5373dd-4aec-4e93-97fb-a639aa096764.jpg?1611169955", 'https://c1.scryfall.com/file/scryfall-cards/border_crop/front/5/c/5c5373dd-4aec-4e93-97fb-a639aa096764.jpg?1614476274'])
master_hash["name_to_url"]["Snow-Covered Plains"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/a/f/afd2730f-878e-47ee-ad2a-73f8fa4e0794.jpg?1610996924", "https://c1.scryfall.com/file/scryfall-cards/border_crop/front/6/f/6fb78693-354a-49d0-a493-430a89c6e6f6.jpg?1610153175"])
master_hash["name_to_url"]["Goldmaw Champion"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/4/4/44d4cd64-8560-48f5-b0bc-f75286c1c91c.jpg?1610644356", "https://c1.scryfall.com/file/scryfall-cards/border_crop/front/4/4/44d4cd64-8560-48f5-b0bc-f75286c1c91c.jpg?1614475605"])
master_hash["name_to_url"]["Giant Ox"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/f/c/fc71d8ca-c613-4534-bc9d-bc1e13202a2c.jpg?1611517846", "https://c1.scryfall.com/file/scryfall-cards/border_crop/front/f/c/fc71d8ca-c613-4534-bc9d-bc1e13202a2c.jpg?1614475479"])
master_hash["name_to_url"]["Warhorn Blast"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/3/f/3fc98aff-edc0-4f78-ae4f-e08735c9e512.jpg?1610999564", "https://c1.scryfall.com/file/scryfall-cards/border_crop/front/3/f/3fc98aff-edc0-4f78-ae4f-e08735c9e512.jpg?1614476515"])
master_hash["name_to_url"]["Depart the Realm"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/2/c/2ce0403d-76ed-4eb0-abdd-74ed28f96137.jpg?1611520838", "https://c1.scryfall.com/file/scryfall-cards/border_crop/front/2/c/2ce0403d-76ed-4eb0-abdd-74ed28f96137.jpg?1614477038"])
master_hash["name_to_url"]["Mists of Littjara"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/6/c/6cfd4944-790f-4835-a8a1-c23ef7047648.jpg?1611077980", 'https://c1.scryfall.com/file/scryfall-cards/border_crop/front/6/c/6cfd4944-790f-4835-a8a1-c23ef7047648.jpg?1614477594'])
master_hash["name_to_url"]["Snow-Covered Mountain"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/5/4/5474e67c-628f-41b0-aa31-3d85a267265a.jpg?1610996923", "https://c1.scryfall.com/file/scryfall-cards/border_crop/front/f/5/f54aa46b-4b7c-4846-bf19-a289ed36c172.jpg?1610153758"])
master_hash["name_to_url"]["Karfell Harbinger"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/4/9/4911016c-b92a-47cc-9553-b424d7be196a.jpg?1610656768", "https://c1.scryfall.com/file/scryfall-cards/border_crop/front/4/9/4911016c-b92a-47cc-9553-b424d7be196a.jpg?1614477526"])
master_hash["name_to_url"]["Wings of the Cosmos"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/d/6/d6b26c95-f90d-43fb-8c99-2a3aa13ac2c6.jpg?1610562354", "https://c1.scryfall.com/file/scryfall-cards/border_crop/front/d/6/d6b26c95-f90d-43fb-8c99-2a3aa13ac2c6.jpg?1614476552"])
master_hash["name_to_url"]["Pilfering Hawk"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/7/6/76ee3829-8dec-4c0f-a6c2-ad47a1c94cfc.jpg?1610996908", "https://c1.scryfall.com/file/scryfall-cards/border_crop/front/7/6/76ee3829-8dec-4c0f-a6c2-ad47a1c94cfc.jpg?1614477750"])
master_hash["name_to_url"]["Frost Augur"].append("https://c1.scryfall.com/file/scryfall-cards/border_crop/front/a/2/a2ae592f-caf6-445a-970b-f8101998e657.jpg?1610996862")
master_hash["name_to_url"]["Valkyrie's Sword"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/9/7/97873c66-6bff-4d79-850c-1e2663098ef4.jpg?1610714064",  'https://c1.scryfall.com/file/scryfall-cards/border_crop/front/9/7/97873c66-6bff-4d79-850c-1e2663098ef4.jpg?1614476433'])
master_hash["name_to_url"]["Behold the Multiverse"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/2/7/27855a38-a682-4f97-ad22-ac625e86faec.jpg?1610586049", "https://c1.scryfall.com/file/scryfall-cards/border_crop/front/2/7/27855a38-a682-4f97-ad22-ac625e86faec.jpg?1614476806"])
master_hash["name_to_url"]["Axgard Braggart"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/4/d/4de5ff64-6fe7-4fc5-be27-cdbaa14545ab.jpg?1611000171", "https://c1.scryfall.com/file/scryfall-cards/border_crop/front/4/d/4de5ff64-6fe7-4fc5-be27-cdbaa14545ab.jpg?1614478895"])
master_hash["name_to_url"]["Disdainful Stroke"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/7/6/7691ac89-f8ba-493e-aa11-5674a783dffb.jpg?1610999654", 'https://c1.scryfall.com/file/scryfall-cards/border_crop/front/7/6/7691ac89-f8ba-493e-aa11-5674a783dffb.jpg?1614477080'])
master_hash["name_to_url"]["Icebind Pillar"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/a/3/a3152f1f-5d3f-4bc4-9b6f-f2983ab3f691.jpg?1611170091", "https://c1.scryfall.com/file/scryfall-cards/border_crop/front/a/3/a3152f1f-5d3f-4bc4-9b6f-f2983ab3f691.jpg?1614477407"])
master_hash["name_to_url"]["Bind the Monster"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/0/8/080be8d1-e453-4250-b917-f0eb26644895.jpg?1611078481", "https://c1.scryfall.com/file/scryfall-cards/border_crop/front/0/8/080be8d1-e453-4250-b917-f0eb26644895.jpg?1614476889"])
master_hash["name_to_url"]["Valor of the Worthy"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/8/3/83632e73-be3f-43fc-932f-45cb1af4f8fb.jpg?1611053713",  'https://c1.scryfall.com/file/scryfall-cards/border_crop/front/8/3/83632e73-be3f-43fc-932f-45cb1af4f8fb.jpg?1614476470'])
master_hash["name_to_url"]["Shimmerdrift Vale"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/f/0/f09d98db-0176-41a7-b99b-ead29876cdab.jpg?1610999466", "https://c1.scryfall.com/file/scryfall-cards/border_crop/front/f/0/f09d98db-0176-41a7-b99b-ead29876cdab.jpg?1612006512"])
master_hash["name_to_url"]["Frostpeak Yeti"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/3/f/3f406ce3-dce1-42d3-b76c-8e8c4b2770cb.jpg?1611000324", "https://c1.scryfall.com/file/scryfall-cards/border_crop/front/3/f/3f406ce3-dce1-42d3-b76c-8e8c4b2770cb.jpg?1614477201"])
master_hash["name_to_url"]["Invoke the Divine"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/7/2/7213d9e1-e9c6-45d8-bb7f-336c41e7b6f7.jpg?1610997029", 'https://c1.scryfall.com/file/scryfall-cards/border_crop/front/7/2/7213d9e1-e9c6-45d8-bb7f-336c41e7b6f7.jpg?1614475645'])
master_hash["name_to_url"]["Usher of the Fallen"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/2/4/24e58478-c6a8-4f86-854a-a489c99bd777.jpg?1610732645", "https://c1.scryfall.com/file/scryfall-cards/border_crop/front/2/4/24e58478-c6a8-4f86-854a-a489c99bd777.jpg?1614476397"])
master_hash["name_to_url"]["Battershield Warrior"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/f/b/fb79dc85-f85a-4846-9ac3-c75c64ea9c0a.jpg?1611205587",  'https://c1.scryfall.com/file/scryfall-cards/border_crop/front/f/b/fb79dc85-f85a-4846-9ac3-c75c64ea9c0a.jpg?1614478927'])
master_hash["name_to_url"]["Draugr Thought-Thief"].append("https://c1.scryfall.com/file/scryfall-cards/border_crop/front/3/1/31da60a2-d4d3-4954-be59-c7ff61fa2905.jpg?1611110658")
master_hash["name_to_url"]["Avalanche Caller"].append("https://c1.scryfall.com/file/scryfall-cards/border_crop/front/8/9/89cef049-6a47-4264-b2bc-b9d291a09c4c.jpg?1610996846")
master_hash["name_to_url"]["Story Seeker"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/e/3/e3dae817-3db1-4edf-86ba-c2c2b238fcf5.jpg?1610999539", "https://c1.scryfall.com/file/scryfall-cards/border_crop/front/e/8/e8e645c8-90ac-4865-b441-e64251d6c9a8.jpg?1614475567", "https://c1.scryfall.com/file/scryfall-cards/border_crop/front/e/3/e3dae817-3db1-4edf-86ba-c2c2b238fcf5.jpg?1614476359"])
master_hash["name_to_url"]["Snow-Covered Island"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/3/b/3bfa5ebc-5623-4eec-89ea-dc187489ee4a.jpg?1610996921", "https://c1.scryfall.com/file/scryfall-cards/border_crop/front/0/d/0dcf8c0a-cf91-42a0-a54e-da5e7c697ee3.jpg?1610153284"])
master_hash["name_to_url"]["Battlefield Raptor"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/3/8/389f0045-218d-41cd-bdca-8a9a0ab1b31b.jpg?1610999580", "https://c1.scryfall.com/file/scryfall-cards/border_crop/front/3/8/389f0045-218d-41cd-bdca-8a9a0ab1b31b.jpg?1614478965"] )
master_hash["name_to_url"]["Tibalt's Trickery"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/d/d/dd921e27-3e08-438c-bec2-723226d35175.jpg?1610391556",  'https://c1.scryfall.com/file/scryfall-cards/border_crop/front/d/d/dd921e27-3e08-438c-bec2-723226d35175.jpg?1613386926'] )
master_hash["name_to_url"]["Codespell Cleric"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/1/3/1316a74e-da55-4668-87f2-98389a90a2a4.jpg?1610586454", "https://c1.scryfall.com/file/scryfall-cards/border_crop/front/1/3/1316a74e-da55-4668-87f2-98389a90a2a4.jpg?1614475315"] )
master_hash["name_to_url"]["Frostpyre Arcanist"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/2/7/276f8abd-4d39-4b50-876f-9c6713bc4ab5.jpg?1610643395", "https://c1.scryfall.com/file/scryfall-cards/border_crop/front/2/7/276f8abd-4d39-4b50-876f-9c6713bc4ab5.jpg?1614477246"] )
master_hash["name_to_url"]["Kardur, Doomscourge"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/9/6/9685e2a0-5573-41bc-a914-f40c3011459b.jpg?1611000522", 'https://c1.scryfall.com/file/scryfall-cards/border_crop/front/9/6/9685e2a0-5573-41bc-a914-f40c3011459b.jpg?1612480876'] )
master_hash["name_to_url"]["Axgard Armory"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/f/9/f9c77d35-8418-48fb-b7d7-7cfa763545c5.jpg?1611000199", 'https://c1.scryfall.com/file/scryfall-cards/border_crop/front/f/9/f9c77d35-8418-48fb-b7d7-7cfa763545c5.jpg?1611926236'] )
master_hash["name_to_url"]["Snow-Covered Forest"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/c/a/ca17acea-f079-4e53-8176-a2f5c5c408a1.jpg?1610996919", "https://c1.scryfall.com/file/scryfall-cards/border_crop/front/f/e/fe241460-f7c1-4ba8-a156-6720b494ac97.jpg?1610153896"] )
master_hash["name_to_url"]["Rune of Sustenance"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/0/e/0e31b716-f325-445a-9098-9ca75d7b35a4.jpg?1611077402", "https://c1.scryfall.com/file/scryfall-cards/border_crop/front/0/e/0e31b716-f325-445a-9098-9ca75d7b35a4.jpg?1614475983"] )
master_hash["name_to_url"]["Stalwart Valkyrie"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/8/9/89ecb530-e429-4b50-a24c-8437614cf224.jpg?1610923714", "https://c1.scryfall.com/file/scryfall-cards/border_crop/front/8/9/89ecb530-e429-4b50-a24c-8437614cf224.jpg?1614476236"] )
master_hash["name_to_url"]["Giant's Amulet"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/a/b/ab8b1ec2-9303-4722-8644-b3bc1a5c387f.jpg?1611000629", 'https://c1.scryfall.com/file/scryfall-cards/border_crop/front/a/b/ab8b1ec2-9303-4722-8644-b3bc1a5c387f.jpg?1614477287'] )
master_hash["name_to_url"]["Inga Rune-Eyes"].append("https://c1.scryfall.com/file/scryfall-cards/border_crop/front/6/2/62a6f095-5beb-42a8-af8f-d40eef27150e.jpg?1610153104" )
master_hash["name_to_url"]["Berg Strider"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/f/3/f3567bdc-450e-4481-9349-a80fe52fe431.jpg?1610996849", "https://c1.scryfall.com/file/scryfall-cards/border_crop/front/f/3/f3567bdc-450e-4481-9349-a80fe52fe431.jpg?1614476852"] )
master_hash["name_to_url"]["Clarion Spirit"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/8/6/86a4c348-1012-4339-960a-c7bc7fd84fbb.jpg?1610566443", 'https://c1.scryfall.com/file/scryfall-cards/border_crop/front/8/6/86a4c348-1012-4339-960a-c7bc7fd84fbb.jpg?1614475278'] )
master_hash["name_to_url"]["Spectral Steel"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/c/f/cf036489-ef9e-40ee-a1bb-24ee37c554f1.jpg?1610585883",  'https://c1.scryfall.com/file/scryfall-cards/border_crop/front/c/f/cf036489-ef9e-40ee-a1bb-24ee37c554f1.jpg?1614476202'] )
master_hash["name_to_url"]["Iron Verdict"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/2/f/2fb38f5b-a2c2-4b06-8c9c-9615475a43e7.jpg?1610586709", "https://c1.scryfall.com/file/scryfall-cards/border_crop/front/2/f/2fb38f5b-a2c2-4b06-8c9c-9615475a43e7.jpg?1614475687"] )
master_hash["name_to_url"]["Shepherd of the Cosmos"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/c/e/ce33c84e-008c-48d8-a8e8-77cd19cc4f1f.jpg?1610586538", 'https://c1.scryfall.com/file/scryfall-cards/border_crop/front/c/e/ce33c84e-008c-48d8-a8e8-77cd19cc4f1f.jpg?1614476118'] )
master_hash["name_to_url"]["Mistwalker"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/9/2/92f64b49-327c-473b-a492-f020a322aed7.jpg?1611170180", 'https://c1.scryfall.com/file/scryfall-cards/border_crop/front/9/2/92f64b49-327c-473b-a492-f020a322aed7.jpg?1614477635'] )
master_hash["name_to_url"]["Glimpse the Cosmos"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/d/c/dc96ebe8-f4b3-4788-a6f5-2a50b6c0d935.jpg?1610066015", 'https://c1.scryfall.com/file/scryfall-cards/border_crop/front/d/c/dc96ebe8-f4b3-4788-a6f5-2a50b6c0d935.jpg?1614477320'] )
master_hash["name_to_url"]["Sigrid, God-Favored"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/a/e/aeec3c1e-e612-4700-888e-300912932552.jpg?1610151617", 'https://c1.scryfall.com/file/scryfall-cards/border_crop/front/a/e/aeec3c1e-e612-4700-888e-300912932552.jpg?1614476164'] )
master_hash["name_to_url"]["Gods' Hall Guardian"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/e/8/e8e645c8-90ac-4865-b441-e64251d6c9a8.jpg?1610289885", "https://c1.scryfall.com/file/scryfall-cards/border_crop/front/e/8/e8e645c8-90ac-4865-b441-e64251d6c9a8.jpg?1614475567"] )
master_hash["name_to_url"]["Reidane, God of the Worthy // Valkmira, Protector's Shield"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/3/6/3606519e-5677-4c21-a34e-be195b6669fa.jpg?1611000242", "https://c1.scryfall.com/file/scryfall-cards/border_crop/front/3/6/3606519e-5677-4c21-a34e-be195b6669fa.jpg?1612724106"] )
master_hash["name_to_url"]["Bound in Gold"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/0/7/074d526a-1eef-4045-bd38-f6d68c4bc4b9.jpg?1610495209", 'https://c1.scryfall.com/file/scryfall-cards/border_crop/front/0/7/074d526a-1eef-4045-bd38-f6d68c4bc4b9.jpg?1614475237'] )
master_hash["name_to_url"]["Divine Gambit"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/6/9/696a8c12-4a1f-4b96-a921-538fa1a2de43.jpg?1610647857", "https://c1.scryfall.com/file/scryfall-cards/border_crop/front/6/9/696a8c12-4a1f-4b96-a921-538fa1a2de43.jpg?1614475355"] )
master_hash["name_to_url"]["Esika's Chariot"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/a/8/a87606cc-fbf0-4e2c-9798-f1c935d0573d.jpg?1610226891", 'https://c1.scryfall.com/file/scryfall-cards/border_crop/front/a/8/a87606cc-fbf0-4e2c-9798-f1c935d0573d.jpg?1612481175'] )
master_hash["name_to_url"]["Kaya's Onslaught"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/f/2/f28e151f-b61b-486f-b7f8-7abde207c442.jpg?1610586077", 'https://c1.scryfall.com/file/scryfall-cards/border_crop/front/f/2/f28e151f-b61b-486f-b7f8-7abde207c442.jpg?1614475732'] )
master_hash["name_to_url"]["Graven Lore"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/e/4/e4386e5c-32b1-4096-a93f-fedd1a4801fc.jpg?1610996879", 'https://c1.scryfall.com/file/scryfall-cards/border_crop/front/e/4/e4386e5c-32b1-4096-a93f-fedd1a4801fc.jpg?1614477364'] )
master_hash["name_to_url"]["Runeforge Champion"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/4/3/4368b087-05a0-4a88-ab43-0cd16446239b.jpg?1611077052", "https://c1.scryfall.com/file/scryfall-cards/border_crop/front/4/3/4368b087-05a0-4a88-ab43-0cd16446239b.jpg?1614476025"] )
master_hash["name_to_url"]["Icebreaker Kraken"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/f/1/f1fdb9bb-09a2-4ff7-bcd4-35ea33c1b752.jpg?1611078087", 'https://c1.scryfall.com/file/scryfall-cards/border_crop/front/f/1/f1fdb9bb-09a2-4ff7-bcd4-35ea33c1b752.jpg?1614477441'] )
master_hash["name_to_url"]["Augury Raven"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/a/9/a9947cfd-91d6-479e-a7f6-2a2050f020f3.jpg?1610586032",  'https://c1.scryfall.com/file/scryfall-cards/border_crop/front/a/9/a9947cfd-91d6-479e-a7f6-2a2050f020f3.jpg?1614476715'] )
master_hash["name_to_url"]["Cosmos Charger"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/4/e/4ec318c6-b718-436f-b9e8-e0c6154e5010.jpg?1610585974", 'https://c1.scryfall.com/file/scryfall-cards/border_crop/front/4/e/4ec318c6-b718-436f-b9e8-e0c6154e5010.jpg?1614476964'] )
master_hash["name_to_url"]["Search for Glory"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/b/6/b65c215d-562d-4c7c-bc9f-b1d741050158.jpg?1610996917",  'https://c1.scryfall.com/file/scryfall-cards/border_crop/front/b/6/b65c215d-562d-4c7c-bc9f-b1d741050158.jpg?1614476078'] )
master_hash["name_to_url"]["Starnheim Unleashed"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/e/a/ea660d42-a441-48d8-98c1-00933cde94a4.jpg?1610586121",  'https://c1.scryfall.com/file/scryfall-cards/border_crop/front/e/a/ea660d42-a441-48d8-98c1-00933cde94a4.jpg?1614476316'] )
master_hash["name_to_url"]["Glorious Protector"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/d/9/d9f08552-7f1a-4fdb-9a70-d99cdbf75910.jpg?1611000537", 'https://c1.scryfall.com/file/scryfall-cards/border_crop/front/d/9/d9f08552-7f1a-4fdb-9a70-d99cdbf75910.jpg?1614475523'] )
master_hash["name_to_url"]["Cyclone Summoner"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/b/2/b2d40071-cadf-48de-a8ed-d55adbfab632.jpg?1610656719", 'https://c1.scryfall.com/file/scryfall-cards/border_crop/front/b/2/b2d40071-cadf-48de-a8ed-d55adbfab632.jpg?1614477000'] )
master_hash["name_to_url"]["Barkchannel Pathway // Tidechannel Pathway"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/b/6/b6de14ae-0132-4261-af00-630bf15918cd.jpg?1608226844", 'https://c1.scryfall.com/file/scryfall-cards/border_crop/front/b/6/b6de14ae-0132-4261-af00-630bf15918cd.jpg?1613329753'] )
master_hash["name_to_url"]["Ascendant Spirit"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/e/a/ea56de43-c0fd-4627-846d-41a962b95f17.jpg?1611798233",  'https://c1.scryfall.com/file/scryfall-cards/border_crop/front/e/a/ea56de43-c0fd-4627-846d-41a962b95f17.jpg?1614476676'] )
master_hash["name_to_url"]["Doomskar"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/1/3/130ee895-1e5e-4f82-bb66-e1275bac75dd.jpg?1611054468", "https://c1.scryfall.com/file/scryfall-cards/border_crop/front/1/3/130ee895-1e5e-4f82-bb66-e1275bac75dd.jpg?1614475398"] )
master_hash["name_to_url"]["Darkbore Pathway // Slitherbore Pathway"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/8/7/87a4e5fe-161f-42da-9ca2-67c8e8970e94.jpg?1608057969", 'https://c1.scryfall.com/file/scryfall-cards/border_crop/front/8/7/87a4e5fe-161f-42da-9ca2-67c8e8970e94.jpg?1612554438'] )
master_hash["name_to_url"]["Rally the Ranks"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/0/3/03595195-3be2-4d18-b5c0-43b2dcc1c0f5.jpg?1610585956", "https://c1.scryfall.com/file/scryfall-cards/border_crop/front/0/3/03595195-3be2-4d18-b5c0-43b2dcc1c0f5.jpg?1614475816"] )
master_hash["name_to_url"]["Resplendent Marshal"].append("https://c1.scryfall.com/file/scryfall-cards/border_crop/front/5/e/5e849bec-2ee6-4c83-83e6-bf9f5543e414.jpg?1611027770" )
master_hash["name_to_url"]["Orvar, the All-Form"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/7/f/7f36775e-9e48-49cc-a771-d58481712edc.jpg?1610997185",  'https://c1.scryfall.com/file/scryfall-cards/border_crop/front/7/f/7f36775e-9e48-49cc-a771-d58481712edc.jpg?1614477709'] )
master_hash["name_to_url"]["Alrund's Epiphany"].append("https://c1.scryfall.com/file/scryfall-cards/border_crop/front/c/9/c94fcb53-a7bd-4a80-a536-9fb0eb24261a.jpg?1610586029" )
master_hash["name_to_url"]["Alrund, God of the Cosmos // Hakka, Whispering Raven"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/5/d/5d131784-c1a3-463e-a37b-b720af67ab62.jpg?1611323243", 'https://c1.scryfall.com/file/scryfall-cards/border_crop/front/5/d/5d131784-c1a3-463e-a37b-b720af67ab62.jpg?1612316166'] )
master_hash["name_to_url"]["Righteous Valkyrie"].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/front/0/2/02fb5f9f-8750-4eb5-a03a-6dacc60e0b90.jpg?1610996474", "https://c1.scryfall.com/file/scryfall-cards/border_crop/front/0/2/02fb5f9f-8750-4eb5-a03a-6dacc60e0b90.jpg?1614475946"] )
master_hash["name_to_url"]['Blightstep Pathway // Searstep Pathway'].append(["https://c1.scryfall.com/file/scryfall-cards/border_crop/back/0/c/0ce39a19-f51d-4a35-ae80-5b82eb15fcff.jpg?1608058149", 'https://c1.scryfall.com/file/scryfall-cards/border_crop/front/0/c/0ce39a19-f51d-4a35-ae80-5b82eb15fcff.jpg?1608058149'] )

master_hash["name_to_color"]["Frost Augur"] = ["U"]
master_hash["name_to_color"]["Draugr Thought-Thief"] = ["U"]
master_hash["name_to_color"]["Avalanche Caller"] = ["U"]
master_hash["name_to_color"]["Inga Rune-Eyes"] = ["U"]
master_hash["name_to_color"]["Alrund's Epiphany"] = ["U"]
master_hash["name_to_color"]["Resplendent Marshal"] = ["W"]

master_hash["name_to_cmc"]["Frost Augur"] = 1.0
master_hash["name_to_cmc"]["Avalanche Caller"] = 2.0
master_hash["name_to_cmc"]["Draugr Thought-Thief"] = 3.0
master_hash["name_to_cmc"]["Inga Rune-Eyes"] = 4.0
master_hash["name_to_cmc"]["Resplendent Marshal"] = 3.0
master_hash["name_to_cmc"]["Alrund's Epiphany"] = 7.0

# master_hash["url_to_name"] = {v: k for k, v in master_hash["name_to_url"].items()}

### All these manutal adjustments unfortunately made things messy, so the below code made sure the output of each "url_to_name" call is a list of urls, rather than a single string

In [ ]:
for key in master_hash["name_to_url"]:
  new_output = []
  for item in master_hash["name_to_url"][key]:
    if (isinstance(item, str)):
      new_output.append(item)
    else:
      for url in item:
        new_output.append(url)
  master_hash["name_to_url"][key] = new_output

In [ ]:
master_hash["name_to_url"]["Righteous Valkyrie"]

['https://c1.scryfall.com/file/scryfall-cards/border_crop/front/0/2/02fb5f9f-8750-4eb5-a03a-6dacc60e0b90.jpg?1614475946',
 'https://c1.scryfall.com/file/scryfall-cards/border_crop/front/0/2/02fb5f9f-8750-4eb5-a03a-6dacc60e0b90.jpg?1614985143',
 'https://c1.scryfall.com/file/scryfall-cards/border_crop/front/0/2/02fb5f9f-8750-4eb5-a03a-6dacc60e0b90.jpg?1610996474',
 'https://c1.scryfall.com/file/scryfall-cards/border_crop/front/0/2/02fb5f9f-8750-4eb5-a03a-6dacc60e0b90.jpg?1614475946']

In [ ]:
for key in master_hash["name_to_url"]:
    if key not in master_hash["name_to_cmc"]:
        print (key)

In [ ]:
def invert_dict(d): 
    inverse = dict() 
    for key in d: 
        # Go through the list that is saved in the dict:
        if (isinstance(d[key], str)):
          if d[key] not in inverse: 
            inverse[d[key]] = key
        else:
          for item in d[key]:
              # Check if in the inverted dict the key exists
              if item not in inverse: 
                  # If not create a new list
                  inverse[item] = key
    return inverse

In [ ]:
master_hash["url_to_name"] = invert_dict(master_hash["name_to_url"])
len(master_hash["url_to_name"]), len(master_hash["name_to_url"])

(542, 305)

In [ ]:
master_hash["name_to_index"] = {}
master_hash["index_to_name"] = {}
array_of_names = []
for key, value in master_hash["name_to_url"].items():
  array_of_names.append(key)
for j in range(len(array_of_names)):
  master_hash["name_to_index"][array_of_names[j]] = j
  master_hash["index_to_name"][j] = array_of_names[j]

In [ ]:
master_hash_save = open("/content/gdrive/MyDrive/Colab Notebooks/17_lands_scrapes/kaldheim/master_hash", "wb")
pickle.dump(master_hash, master_hash_save)
master_hash_save.close()

In [ ]:
len(master_hash["name_to_index"])

305

### The code below was called once to create names for the card images saved in the image folder.  It is not used again

In [ ]:
# import urllib.request

In [ ]:
# urllib.request.urlretrieve("https://c1.scryfall.com/file/scryfall-cards/border_crop/front/0/c/0ce39a19-f51d-4a35-ae80-5b82eb15fcff.jpg?1608058149", "/content/gdrive/MyDrive/Colab Notebooks/17_lands_scrapes/kaldheim/card_images/test.jpg")

In [ ]:
# for name in master_hash["name_to_url"]:
#   if (isinstance(master_hash["name_to_url"][name], str)):
#     card_name = copy.deepcopy(name)
#     card_name = card_name.replace("/", "")
#     urllib.request.urlretrieve(master_hash["name_to_url"][name], "/content/gdrive/MyDrive/Colab Notebooks/17_lands_scrapes/kaldheim/card_images/" + card_name + ".jpg")
#   else: 
#     card_name = copy.deepcopy(name)
#     card_name = card_name.replace("/", "")
#     urllib.request.urlretrieve(master_hash["name_to_url"][name][0], "/content/gdrive/MyDrive/Colab Notebooks/17_lands_scrapes/kaldheim/card_images/" + card_name + ".jpg")